In [1]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
import sqlite3


In [2]:

# Koneksi ke SQLite database
conn = sqlite3.connect('dummy_data.db')
cursor = conn.cursor()

# Mengambil data transaksi
transaksi_df = pd.read_sql_query('SELECT * FROM transaksi', conn)
transaksi_df

# Mengambil data promo
promo_df = pd.read_sql_query('SELECT * FROM promo', conn)
promo_df



,id,nama_promo,category,merchant
0,1,Promo Diskon Minuman,minuman,kopi kenangan
1,2,Promo Buy 1 Get 1 Minuman,minuman,fore coffee
2,3,Promo Pulsa Murah,tagihan,pulsa telkomsel
3,4,Promo Cuci Mobil Hemat,transportasi,cuco mobil andi
4,5,Promo Ganti Oli Gratis,transportasi,bengkel mamad
5,6,Promo Diskon 20%,minuman,sakara kopi bali
6,7,Promo TGIF,minuman,starbucks
7,8,Promo Buy 2 Get 3,minuman,tomoro coffee


In [17]:

# Close the SQLite connection
conn.close()

In [3]:
transaksi_df.head()

,id,account_number,merchant_name,category,date
0,1,100013129837,kopi kenangan,minuman,2024-08-01
1,2,100013129837,fore coffee,minuman,2024-08-02
2,3,100013129837,pulsa telkomsel,tagihan,2024-08-03
3,4,100013129837,cuci mobil andi,transportasi,2024-08-04
4,5,100013129837,bengkel mamad,transportasi,2024-08-05


In [4]:
promo_df

,id,nama_promo,category,merchant
0,1,Promo Diskon Minuman,minuman,kopi kenangan
1,2,Promo Buy 1 Get 1 Minuman,minuman,fore coffee
2,3,Promo Pulsa Murah,tagihan,pulsa telkomsel
3,4,Promo Cuci Mobil Hemat,transportasi,cuco mobil andi
4,5,Promo Ganti Oli Gratis,transportasi,bengkel mamad
5,6,Promo Diskon 20%,minuman,sakara kopi bali
6,7,Promo TGIF,minuman,starbucks
7,8,Promo Buy 2 Get 3,minuman,tomoro coffee


In [5]:
# Create a pivot table (user-item matrix)
user_item_matrix = transaksi_df.pivot_table(index='account_number', columns='merchant_name', aggfunc='size', fill_value=0)

# Save the columns (merchant names) and index (account IDs) for later
merchants = user_item_matrix.columns
accounts = user_item_matrix.index

# Convert the matrix to a dense NumPy array with floating-point data type
user_item_matrix = user_item_matrix.to_numpy(dtype=float)

# Determine the value of k based on the shape of the matrix
num_accounts, num_merchants = user_item_matrix.shape
k = min(num_accounts, num_merchants) - 1  # k must be < min(A.shape)

# Apply SVD with the correct value of k
U, sigma, Vt = svds(user_item_matrix, k=k)

# Convert sigma to a diagonal matrix
sigma = np.diag(sigma)

In [6]:

# Reconstruct the approximate matrix
predicted_matrix = np.dot(np.dot(U, sigma), Vt)

# Convert back to a DataFrame for easier interpretation
predicted_df = pd.DataFrame(predicted_matrix, index=accounts, columns=merchants)

# Display the predicted matrix
print("Predicted User-Item Interaction Matrix:")
print(predicted_df)

Predicted User-Item Interaction Matrix:
merchant_name   bengkel mamad  cuci mobil andi  flash coffee  fore coffee  \
account_number                                                              
100013129837         1.717007         2.203547      1.588313     1.101773   
109010231029         1.357847         1.742614      1.256074     0.871307   

merchant_name   kopi kenangan  pulsa telkomsel  
account_number                                  
100013129837         3.434013         1.588313  
109010231029         2.715694         1.256074  


In [7]:
# Function to recommend promotions based on the predicted matrix
def recommend_promos(account_number, num_recommendations=3):
    # Get the predicted scores for the given account
    user_predictions = predicted_df.loc[account_number]
    
    # Sort merchants by predicted score
    sorted_merchants = user_predictions.sort_values(ascending=False)
    
    # Recommend promos that match top merchants
    recommendations = []
    for merchant in sorted_merchants.index[:num_recommendations]:
        promos = promo_df[promo_df['merchant'] == merchant]
        if not promos.empty:
            recommendations.extend(promos['nama_promo'].tolist())
    
    return recommendations

In [8]:

# Example recommendation for a user
account_number = '100013129837'
recommended_promos = recommend_promos(account_number)
print(f"Recommended promotions for account {account_number}: {recommended_promos}")

Recommended promotions for account 100013129837: ['Promo Diskon Minuman', 'Promo Ganti Oli Gratis']
